In [1833]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [1834]:
pd.set_option('display.max_row',111)
pd.set_option('display.max_column',111)

In [1835]:

train_df=pd.read_csv('C:/Users/hp/Desktop/bac-mauritanie-2022-predictive-modeling-challeng (1)/train_set.csv',sep=',')
test_df=pd.read_csv('C:/Users/hp/Desktop/bac-mauritanie-2022-predictive-modeling-challeng (1)/test_set.csv',sep=',')

In [1836]:
string_vars = train_df.select_dtypes(include='object').columns.tolist()
print(string_vars)


['Nom', 'Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre', 'Etablissement', 'Decision', 'Willaya', 'moughataa', 'DateNaissance', 'ID']


In [1837]:
train_df['DateNaissance']


0        12 ديسمبر 1998
1        26 ديسمبر 1991
2           8 مايو 1995
3          24 avr. 1999
4           10 mai 2001
              ...      
23889      15 مارس 2001
23890      31 déc. 1995
23891      6 févr. 1999
23892      28 août 2000
23893    21 ديسمبر 1997
Name: DateNaissance, Length: 23894, dtype: object

In [1839]:
train_df["Reussite"] = train_df.apply(lambda row: "Reussi" if row["moyeneGeneral"] >= 10 else "Proche" if row["moyeneGeneral"] >= 7 else "Loin", axis=1)

In [1843]:
train_df.shape

(23894, 22)

In [1786]:
test_df.shape

(15930, 12)

In [1849]:
import pandas as pd

# define the replacement dictionary
remplacements = {'janv.' : 'Jan',
                 'févr.' : 'Feb',
                 'mars' : 'Mar',
                 'avr.' : 'Apr',
                 'mai' : 'May',
                 'juin' : 'Jun',
                 'juil.' : 'Jul',
                 'août' : 'Aug',
                 'sept.' : 'Sep',
                 'oct.' : 'Oct',
                 'nov.' : 'Nov',
                 'déc.' : 'Dec'}

# replace the abbreviated month names with their English names
train_df['DateNaissance'] = train_df['DateNaissance'].replace(remplacements, regex=True)

# convert the DateNaissance column to datetime format
#train_df['DateNaissance'] = pd.to_datetime(train_df['DateNaissance'], format='%d %b %Y', errors='coerce')


In [1850]:
import pandas as pd

# define the replacement dictionary
remplacements = {'janv.' : 'Jan',
                 'févr.' : 'Feb',
                 'mars' : 'Mar',
                 'avr.' : 'Apr',
                 'mai' : 'May',
                 'juin' : 'Jun',
                 'juil.' : 'Jul',
                 'août' : 'Aug',
                 'sept.' : 'Sep',
                 'oct.' : 'Oct',
                 'nov.' : 'Nov',
                 'déc.' : 'Dec'}

# replace the abbreviated month names with their English names
test_df['DateNaissance'] = test_df['DateNaissance'].replace(remplacements, regex=True)

# convert the DateNaissance column to datetime format
#train_df['DateNaissance'] = pd.to_datetime(train_df['DateNaissance'], format='%d %b %Y', errors='coerce')


In [1851]:
print(train_df['DateNaissance'].isnull().sum())

0


In [1852]:
test_df.dtypes.value_counts()

object     10
int64       1
float64     1
dtype: int64

# Conversion de la variable cible Decision

In [1853]:
train_df['Decision'].value_counts()

Ajourné                                  18966
Admis                                     1973
Sessionnaire                              1530
Abscent                                   1068
Examen annulé à cause du Téléphone         351
Examen annulé à cause du comportement        6
Name: Decision, dtype: int64

In [1854]:

train_df['Decision'] = train_df['Decision'].replace({'Sessionnaire': 'Ajourné', 'Abscent': 'Ajourné','Examen annulé à cause du Téléphone':'Ajourné','Examen annulé à cause du comportement':'Ajourné'})
     

In [1855]:
train_df['Decision'].value_counts()

Ajourné    21921
Admis       1973
Name: Decision, dtype: int64

# Test chi deux pour etudier les liason entre la variable cible et les autres features categorielles

In [1856]:
contingency_table = pd.crosstab(train_df['Willaya'], train_df['Decision'])

In [1857]:
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(contingency_table)
print('Statistique du chi-carré :', chi2)
print('P-valeur :', p)
print('Degrés de liberté :', dof)
print('Tableau de contingence attendu :')
print(expected)


Statistique du chi-carré : 711.1552558446579
P-valeur : 1.9809041192683006e-135
Degrés de liberté : 23
Tableau de contingence attendu :
[[  54.74591948  608.25408052]
 [  77.20578388  857.79421612]
 [  63.66380681  707.33619319]
 [  71.17795262  790.82204738]
 [  75.96718842  844.03281158]
 [  67.21444714  746.78555286]
 [  77.94894116  866.05105884]
 [  60.27831255  669.72168745]
 [  51.6081443   573.3918557 ]
 [  66.63643593  740.36356407]
 [  34.59809994  384.40190006]
 [  32.94663932  366.05336068]
 [  89.34401942  992.65598058]
 [ 103.21628861 1146.78371139]
 [ 107.92295137 1199.07704863]
 [ 122.7035239  1363.2964761 ]
 [  94.87641249 1054.12358751]
 [ 112.05160291 1244.94839709]
 [ 108.50096258 1205.49903742]
 [ 109.49183896 1216.50816104]
 [  98.50962585 1094.49037415]
 [ 106.35406378 1181.64593622]
 [  88.51828911  983.48171089]
 [  97.51874948 1083.48125052]]


In [1858]:



from dateutil.parser import parse

# Iterate over the DateNaissance column and parse each date
dates = []
for date_str in train_df['DateNaissance']:
    try:
        date = parse(date_str)
        dates.append(date)
    except:
        dates.append(None)

# Replace the DateNaissance column with the parsed dates
train_df['DateNaissance'] = dates


In [1859]:



from dateutil.parser import parse

# Iterate over the DateNaissance column and parse each date
dates = []
for date_str in test_df['DateNaissance']:
    try:
        date = parse(date_str)
        dates.append(date)
    except:
        dates.append(None)

# Replace the DateNaissance column with the parsed dates
test_df['DateNaissance'] = dates


In [1860]:
train_df['DateNaissance'] = pd.to_datetime(train_df['DateNaissance'], errors='coerce')
train_df['year'] = train_df['DateNaissance'].dt.year
train_df['month'] = train_df['DateNaissance'].dt.month
train_df['day'] = train_df['DateNaissance'].dt.day
#__________________________________


In [1861]:
test_df['DateNaissance'] = pd.to_datetime(test_df['DateNaissance'], errors='coerce')
test_df['year'] = test_df['DateNaissance'].dt.year
test_df['month'] = test_df['DateNaissance'].dt.month
test_df['day'] = test_df['DateNaissance'].dt.day
#__________________________________


# Creation de la variable Age

In [1862]:
import datetime

train_df['Age'] = 2021 - train_df['year']


In [1863]:
import datetime

test_df['Age'] = 2021 - test_df['year']


In [1864]:
print(train_df[['year','month','day']].isnull().sum())

year     0
month    0
day      0
dtype: int64


In [1865]:
print(test_df[['year','month','day']].isnull().sum())

year     0
month    0
day      0
dtype: int64


In [1866]:
print(train_df.columns)

Index(['Unnamed: 0', 'Nom', 'Serie,x', 'GroupeAnonymat', 'LieuNaissance',
       'Centre', 'Etablissement', 'moyeneGeneral', 'Decision', 'Note1',
       'Note2', 'Note3', 'Note4', 'Note5', 'Note6', 'Note7', 'Note8',
       'Willaya', 'moughataa', 'DateNaissance', 'ID', 'Reussite', 'year',
       'month', 'day', 'Age'],
      dtype='object')


# Elimination des Variables non necessaire

In [1867]:
train_df.drop('DateNaissance', axis=1, inplace=True)
test_df.drop('DateNaissance', axis=1, inplace=True)
train_df.drop(['Note1', 'Note2', 'Note3', 'Note4', 'Note5', 'Note6', 'Note7', 'Note8'], axis=1,inplace=True)
train_df.drop('Nom', axis=1, inplace=True)
#train_df.drop('GroupeAnonymat', axis=1, inplace=True)
train_df.drop('Reussite', axis=1, inplace=True)

In [1868]:
train_df.drop('moyeneGeneral', axis=1, inplace=True)

# Encodage

In [1869]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder(unknown_value='unknown')
# Convert the target variable to binary
train_df['Decision'] = train_df['Decision'].apply(lambda x: 1 if x == 'Admis' else 0)
# Encode categorical variables
le = LabelEncoder()
#train_df["Nom"]=le.fit_transform(train_df["Nom"])
train_df["Serie,x"] = le.fit_transform(train_df["Serie,x"])
train_df["GroupeAnonymat"] = le.fit_transform(train_df["GroupeAnonymat"])
train_df["LieuNaissance"] = le.fit_transform(train_df["LieuNaissance"])
train_df["Centre"] = le.fit_transform(train_df["Centre"])
train_df["Etablissement"] = le.fit_transform(train_df["Etablissement"])
train_df["Willaya"] = le.fit_transform(train_df["Willaya"])
train_df["moughataa"] = le.fit_transform(train_df["moughataa"])
#train_df["Decision"] = le.fit_transform(train_df["Decision"])
train_df["ID"] = le.fit_transform(train_df["ID"])
#train_df["Reussite"]=le.fit_transform(train_df["Reussite"])
# Define X and y)
#X_train = train_df.drop(["Decision"], axis=1)
#y_train = train_df["Decision"]



In [1870]:

# Encode categorical variables
#test_df["Nom"]=le.fit_transform(test_df["Nom"])
test_df["Serie,x"] = le.fit_transform(test_df["Serie,x"])
test_df["GroupeAnonymat"] = le.fit_transform(test_df["GroupeAnonymat"])
test_df["LieuNaissance"] = le.fit_transform(test_df["LieuNaissance"])
test_df["Centre"] = le.fit_transform(test_df["Centre"])
test_df["Etablissement"] = le.fit_transform(test_df["Etablissement"])
test_df["Willaya"] = le.fit_transform(test_df["Willaya"])
test_df["moughataa"] = le.fit_transform(test_df["moughataa"])
test_df["ID"] = le.fit_transform(test_df["ID"])

# Define X_test
X_test = test_df

In [1765]:
train_df['Decision'].value_counts()


0    21921
1     1973
Name: Decision, dtype: int64

In [1871]:
class_counts = train_df['Decision'].value_counts()
desired_class_counts = {0: class_counts.min(), 1: class_counts.min()}


# xGboost 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Define the features and target variable
features = ['Serie,x', 'Centre', 'Etablissement', 'Willaya', 'moughataa','Age']
target = 'Decision'



import xgboost as xgb

# Split the train set into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
X_train = train_df[features]
y_train = train_df[target]
X_val = val_df[features]
y_val = val_df[target]
model = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=5, subsample=0.8, colsample_bytree=0.8, objective='binary:logistic', random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred_proba = model.predict_proba(X_val)[:, 1]

# Calculate ROC AUC score on the validation set
auc_score = roc_auc_score(y_val, y_val_pred_proba)

print("The ROC AUC score on the validation set is %.2f%%" % (auc_score*100))

# Add the predictions to the test dataset
test_df['prediction'] = y_test_pred_proba



#test_df["ID"] = le.inverse_transform(test_df["ID"])
# Save the predictions to a CSV file
f=pd.read_csv("C:/Users/hp/Desktop/bac-mauritanie-2022-predictive-modeling-challeng (1)/submission_template.csv")

submission_df = pd.DataFrame({"Id": f["Id"], "Predicted": test_df['prediction']})
# Save the predictions to a CSV file


dt = submission_df.to_csv("C:/Users/hp/Desktop/bac-mauritanie-2022-predictive-modeling-challeng (1)/submission_template.csv", index=False)

In [1570]:
f=pd.read_csv("C:/Users/hp/Desktop/bac-mauritanie-2022-predictive-modeling-challeng (1)/submission_template.csv")
f.head()

,Id,Predicted
0,ID_12,0.048780
1,ID_14,0.027285
2,ID_15,0.027285
3,ID_22,0.020324
4,ID_23,0.008323


# Logistic Regressiom

In [1885]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

# Define the features and target variable
features = ["Serie,x", "Centre", "Etablissement", "Willaya", "moughataa","Age"]
target = "Decision"

# Split the train set into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Create logistic regression pipeline
model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=10000))

# Train model on training data
model.fit(train_df[features], train_df[target])

# Make predictions on validation set
probas = model.predict_proba(val_df[features])[:, 1]

# Compute the ROC AUC score on the validation data
auc_score = roc_auc_score(val_df[target], probas)

print("The ROC AUC score on the validation set is %.2f%%" % (auc_score*100))

# Make predictions on the test data
X_test = test_df[features]

y_test_pred_proba = model.predict_proba(X_test)[:, 1]

# Add the predictions to the test dataset
test_df['prediction'] = y_test_pred_proba

# Save the predictions to a CSV file
f=pd.read_csv("C:/Users/hp/Downloads/bac-mauritanie-2022-predictive-modeling-challeng (2)/submission_logistic2.csv")

submission_df = pd.DataFrame({"Id": f["Id"], "Predicted": test_df['prediction']})

# Save the predictions to a CSV file
dt = submission_df.to_csv("C:/Users/hp/Downloads/bac-mauritanie-2022-predictive-modeling-challeng (2)/submission_logistic2.csv", index=False)


The ROC AUC score on the validation set is 69.54%


In [1886]:
f.head()


,Id,Predicted
0,ID_12,0.010147
1,ID_14,0.006578
2,ID_15,0.006973
3,ID_22,0.004766
4,ID_23,0.002674


# RandomForest

In [1874]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score as acc_score
# Define the features and target variable
features = ["Serie,x", "Centre", "Etablissement", "Willaya", "moughataa","Age"]
target = "Decision"

# Compute class weights based on the distribution of classes in the training data
class_weights = compute_class_weight('balanced', classes=[0, 1], y=train_df[target])
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Create the random forest model
model = RandomForestClassifier(n_estimators=100, class_weight=class_weight_dict)

# Fit the model to the training data
model.fit(train_df[features], train_df[target])

# Make predictions on the validation data
y_val_pred = model.predict(val_df[features])

# Compute the accuracy score on the validation data
val_accuracy_score = acc_score(val_df[target], y_val_pred)


print("The accuracy score on the validation set is %.2f%%" % (val_accuracy_score*100))

# Make predictions on the test data
X_test = test_df[features]

y_test_pred_proba = model.predict_proba(X_test)[:, 1]

# Add the predictions to the test dataset
test_df['prediction'] = y_test_pred_proba

# Save the predictions to a CSV file
f=pd.read_csv("C:/Users/hp/Downloads/bac-mauritanie-2022-predictive-modeling-challeng/submission_rf.csv")

submission_df = pd.DataFrame({"Id": f["Id"], "Predicted": test_df['prediction']})
# Save the predictions to a CSV file
dt = submission_df.to_csv("C:/Users/hp/Downloads/bac-mauritanie-2022-predictive-modeling-challeng/submission_rf.csv", index=False)


The accuracy score on the validation set is 82.68%


In [1875]:
f.head()

,Id,Predicted
0,ID_12,0.079500
1,ID_14,0.018333
2,ID_15,0.018333
3,ID_22,0.026667
4,ID_23,0.020000
